In [15]:
import pandas as pd

In [23]:
data = {
    "S. No.": [1, 2, 3],
    "Product Name": ["SKU1", "SKU2", "SKU3"],
    "Input Image Urls": [['https://drive.google.com/file/d/1Euim_lg9U-bpzPCzSnctN1rXWooiV4EO/view?usp=drive_link','https://drive.google.com/file/d/13ow16cw2PSBlk99nuoFlCyqHJ9hWNgjN/view?usp=drive_link','https://drive.google.com/file/d/1LIjMTllHX9ztK2F3DWVaMNtCCrGet7_K/view?usp=drive_link'], 
                         ['https://drive.google.com/file/d/1nYSl4WooBuaOMb7HOfq1HmRIFywtQulX/view?usp=drive_link','https://drive.google.com/file/d/1fH1mysu5YRKSn7ERjfvsmw2pGLMWY_Xx/view?usp=drive_link','https://drive.google.com/file/d/1DPdmJ0m5BwXO5N4zVuMKlbJMPMgBXEU_/view?usp=drive_link'],
                          ['https://drive.google.com/file/d/15_aFXWWrbUGzEnfO8TxdU0j4ub204ZsH/view?usp=drive_link','https://drive.google.com/file/d/1YJRu6M07SSXy4qeXofnWXH8KCZqlBoU8/view?usp=drive_link','https://drive.google.com/file/d/10_621W1uHlfG9eqpyo-NS0JMXMOVyunY/view?usp=drive_link']],
    }

In [24]:
df=pd.DataFrame(data)

In [26]:
df.head()

,S. No.,Product Name,Input Image Urls
0,1,SKU1,[https://drive.google.com/file/d/1Euim_lg9U-bp...
1,2,SKU2,[https://drive.google.com/file/d/1nYSl4WooBuaO...
2,3,SKU3,[https://drive.google.com/file/d/15_aFXWWrbUGz...


In [17]:
# df.remove(columns=[''],).head()

In [35]:
df=pd.read_csv('data/data.csv')

In [2]:
import requests
from io import BytesIO
from PIL import Image

In [3]:
import pandas as pd
import json
import ast

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

In [5]:
def authenticate_drive():
    """Authenticate and return a Google Drive instance"""
    scope = ["https://www.googleapis.com/auth/drive"]
    credentials = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    
    gauth = GoogleAuth()
    gauth.credentials = credentials
    return GoogleDrive(gauth)


def upload_to_drive(filename="compressed.jpg"):
    """Upload image data to Google Drive and return the file's public URL"""
    drive = authenticate_drive()
    img_file = drive.CreateFile({'title': filename})
    img_file.SetContentFile(filename)
    img_file.Upload()
    
    # Make file public
    img_file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
    
    return f"https://drive.google.com/uc?id={img_file['id']}"

In [7]:
# file_id = image_url.split("id=")[1]
# response = requests.get(image_url)
# if response.status_code == 200:
#     img = Image.open(BytesIO(response.content))
#     if img.mode in ("RGBA", "P"):
#         img = img.convert("RGB")
#     output_filename = f'compressed-{file_id}'
#     img.save(f'compressed-{file_id}', format="JPEG", quality=50)
#     drive_link = upload_to_drive(output_filename)

In [13]:

df['Processed Image Urls']='processed_url'
for i,row in df.iterrows():
    a=ast.literal_eval(row['Input Image Urls'])
    # print(a)
    b=[]
    for j,image_url in enumerate(a):
        file_id = image_url.split("id=")[1]
        response = requests.get(image_url)
        if response.status_code != 200:
            print('issue with getting image')
        img = Image.open(BytesIO(response.content))
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
        output_filename = f'compressed-{file_id}'
        img.save(f'compressed-{file_id}', format="JPEG", quality=50)
        drive_link = upload_to_drive(output_filename)
        b.append(drive_link)
    df.loc[i, 'Processed Image Urls'] = str(b)

In [15]:
df.head()

,Unnamed: 0,S. No.,Product Name,Input Image Urls,Processed Image Urls
0,0,1,SKU1,['https://drive.google.com/uc?export=download&...,['https://drive.google.com/uc?id=1SljV4-n671vY...
1,1,2,SKU2,['https://drive.google.com/uc?export=download&...,['https://drive.google.com/uc?id=1lGZhHA9GcNho...
2,2,3,SKU3,['https://drive.google.com/uc?export=download&...,['https://drive.google.com/uc?id=1RHvWNmBSkxp8...


In [12]:
df.iloc[2]['Input Image Urls']

"['https://drive.google.com/uc?export=download&id=15_aFXWWrbUGzEnfO8TxdU0j4ub204ZsH', 'https://drive.google.com/uc?export=download&id=1YJRu6M07SSXy4qeXofnWXH8KCZqlBoU8', 'https://drive.google.com/uc?export=download&id=10_621W1uHlfG9eqpyo-NS0JMXMOVyunY']"

In [ ]:
import requests

url = "https://telegra.ph/upload"
files = {"file": open("compressed-1DPdmJ0m5BwXO5N4zVuMKlbJMPMgBXEU_.jpg", "rb")}
response = requests.post(url, files=files)
print("https://telegra.ph" + response.json()[0]["src"])  # Direct image link

https://files.catbox.moe/gq782p.jpg
